In [9]:
#import df
import pandas as pd
import numpy as np
df = pd.read_csv("C:\\Users\\19044\\Downloads\\pj1.csv")
df_1 = pd.read_csv("C:\\Users\\19044\\Downloads\\pj1.csv")
df_cost = pd.read_csv("C:\\Users\\19044\\Downloads\\pj1.csv")
df_cost

,Unnamed: 0,to,Mexico,Canada,Chile,Frankfurt,Austin,Japan
0,from,capacity/demand,3.000,2.60,16.000,20.00000,26.40000,11.9000
1,Mexico,22,92.630,104.03,145.945,112.42985,107.79835,112.1878
2,Canada,3.7,160.200,93.25,148.875,113.60875,103.44625,111.8450
3,Chile,4.5,186.696,122.31,112.310,135.97945,127.76395,133.3486
4,Frankfurt,47,127.344,84.84,122.510,73.34000,87.84030,91.0404
5,Austin,18.5,152.640,95.15,144.725,107.61925,89.15000,106.9990
6,Japan,5,252.784,162.24,236.360,177.61780,168.95580,149.2400


In [10]:
#add capacity constrains
# q2
#df

# # q3
#for i in range(1,7):
    #df.iloc[i,1] = float(df.iloc[i,1])*0.85
#df

# # q4
for i in range(1,7):
    df.iloc[i,1] = float(df.iloc[i,1])*0.9
df.iloc[3,1]=0
df

,Unnamed: 0,to,Mexico,Canada,Chile,Frankfurt,Austin,Japan
0,from,capacity/demand,3.000,2.60,16.000,20.00000,26.40000,11.9000
1,Mexico,19.8,92.630,104.03,145.945,112.42985,107.79835,112.1878
2,Canada,3.33,160.200,93.25,148.875,113.60875,103.44625,111.8450
3,Chile,0,186.696,122.31,112.310,135.97945,127.76395,133.3486
4,Frankfurt,42.3,127.344,84.84,122.510,73.34000,87.84030,91.0404
5,Austin,16.65,152.640,95.15,144.725,107.61925,89.15000,106.9990
6,Japan,4.5,252.784,162.24,236.360,177.61780,168.95580,149.2400


In [11]:
def getIndexes(dfObj, value):
    listOfPos = []
    result = dfObj.isin([value])
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    return listOfPos

In [12]:
#create table for optimized result
df_value = df_cost
df_value.iloc[:] = 0
df_value 

,Unnamed: 0,to,Mexico,Canada,Chile,Frankfurt,Austin,Japan
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0


In [13]:
def getOpportunityCost(data):
    x = data
    x = x[x.notnull()]
    y = x[1:].sort_values().values.tolist()
    if x[0] == 0:
        opp = float("NAN")
    else:
        if len(y)> 1:
            opp = y[1]-y[0]
        else:
            opp = 0
    return opp

In [14]:
#current total demand
total_demand = 0
for k in range(2,8):
    total_demand += df.iloc[0].iat[k]
#start optimization
while total_demand > 0: 
#find highest opportunity cost
    op_cost_to = []
    for a in range(2,len(df_1.columns)):
        op_cost_to.append(getOpportunityCost(df_1.iloc[:,a]))
    op_cost_from = []
    for b in range(1,len(df_1)):
        op_cost_from.append(getOpportunityCost(df_1.iloc[b,1:]))
    op_list = op_cost_to + op_cost_from
    #get index of the next best route
    ind = op_list.index(np.nanmax(op_list))
    if ind <6:
        min_cost = np.nanmin(df_1.iloc[1:,ind+2])
    else:
        min_cost = np.nanmin(df_1.iloc[ind-5,2:])
    min_pos = getIndexes(df_1, min_cost)
    #get demand and capacity for the route
    demand = float(df.loc[0,min_pos[0][1]])
    capacity = float(df.iloc[min_pos[0][0],1])
    #assign quantity to the route
    if demand >= capacity:
        df_value.loc[min_pos[0]] = capacity
        df.loc[0,min_pos[0][1]] = demand - capacity
        df.iloc[min_pos[0][0],1] = 0
    else:
        df_value.loc[min_pos[0]] = demand
        df.iloc[min_pos[0][0],1] = capacity - demand
        df.loc[0,min_pos[0][1]] = 0
    #update demand and capacity
    df_1.iloc[1:,1] = df.iloc[1:,1]
    df_1.iloc[0,2:] = df.iloc[0,2:]
    #update column/row to 0 if capacity/demand = 0
    df_1.loc[1:,(df_1.iloc[0,:]==0).values.tolist()]=float("NAN")
    df_1.loc[(df_1.iloc[:,1]==0).values.tolist(),2:]=float("NAN")
    #get current total demand
    total_demand = 0
    for k in range(2,8):
        total_demand += df.iloc[0].iat[k]

C:\Users\19044\AppData\Local\Temp\ipykernel_7096\3294120382.py:39: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df_1.loc[(df_1.iloc[:,1]==0).values.tolist(),2:]=float("NAN")


In [15]:
df_value

,Unnamed: 0,to,Mexico,Canada,Chile,Frankfurt,Austin,Japan
0,0,0,0,0.0,0,0,0.00,0.0
1,0,0,3,0.0,0,0,9.02,1.1
2,0,0,0,2.6,0,0,0.73,0.0
3,0,0,0,0.0,0,0,0.00,0.0
4,0,0,0,0.0,16,20,0.00,6.3
5,0,0,0,0.0,0,0,16.65,0.0
6,0,0,0,0.0,0,0,0.00,4.5


In [16]:
#calculate total cost
df_Q1 = df_value*df
df_Q1 = df_Q1.sum(axis = 1)
total_cost = 0
for i in range(len(df_Q1)):
    total_cost += df_Q1[i]
total_cost*10

C:\Users\19044\AppData\Local\Temp\ipykernel_7096\1725665759.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_Q1 = df_Q1.sum(axis = 1)


78480.454795